# Tutorial showing the integration between the QDAC2 and the OPX to perform voltage sweeps using the QCoDeS framework

In this tutorial you will learn how to combine the QDAC2 with the OPX to perform n-dimensional voltage scans efficiently.

The trick here is to use a digital marker from the OPX to trigger the QDAC and make it step the output voltage of a given channel across a pre-loaded voltage list. Please note that the size of this list is limited to 65536 items per channel. To this purpose, an element called "qdac_trigger1" was created in the config and a digital pulse was defined. You can read more about digital waveform manipulations here: https://docs.quantum-machines.co/0.1/qm-qua-sdk/docs/Introduction/qua_overview/?h=digi#digital-waveform-manipulations

In this case, the latency induced by the communication between the control PC and the instruments and by transferring the data is suppressed since everything is controlled by the OPX pulse processor. The experimental runtime is then given by the QDAC bandwidth and the integration time. 

In the following examples, the OPX is programmed using its QCoDeS driver. You can learn more about its usage in the github (https://github.com/qua-platform/py-qua-tools/tree/main/qualang_tools/external_frameworks/qcodes) and usage examples (https://github.com/qua-platform/py-qua-tools/tree/main/examples/Qcodes_drivers/basic-driver).
The following can also be adapted to work without controlling the OPX through qcodes so that they match your current framework.

In [ ]:
import os
import time
import qcodes as qc
from qm.qua import *
from qcodes import initialise_or_create_database_at, load_or_create_experiment
from qcodes.utils.dataset.doNd import do1d, do0d
from qualang_tools.external_frameworks.qcodes.opx_driver import OPX
from qcodes_contrib_drivers.drivers.QDevil import QDAC2
from importlib import reload
from configuration import *

## QCoDeS set-up

In [ ]:
db_name = "QM_demo.db"  # Database name
sample_name = "demo"  # Sample name
exp_name = "OPX_QDAC2_integration"  # Experiment name

# Initialize qcodes database
db_file_path = os.path.join(os.getcwd(), db_name)
qc.config.core.db_location = db_file_path
initialise_or_create_database_at(db_file_path)
# Initialize the qcodes station to which instruments will be added
station = qc.Station()

In [ ]:
# Create the OPX instrument class
opx_instrument = OPX(config, name="OPX_demo", host="127.0.0.1", cluster_name="my_cluster")
# Add the OPX instrument to the qcodes station
station.add_component(opx_instrument)

In [ ]:
qdac2_ip = "127.0.0.1"
# Create the QDAC2 instrument class
qdac2 = QDAC2.QDac2("QDAC", visalib="@py", address=f"TCPIP::{qdac2_ip}::5025::SOCKET")
# Add the QDAC2 instrument to the qcodes station
station.add_component(qdac2)

In [ ]:
# Used to reload the configuration in order to force Jupyter to upload the existing variables
import configuration

reload(configuration)
from configuration import *

opx_instrument.config = config
opx_instrument.qm = opx_instrument.qmm.open_qm(config)

## Example experiments:
1. 1D voltage sweep performed by the QDAC using do1d
2. 1D voltage sweep performed by the QDAC triggered by the OPX using do0d
3. 2D voltage sweep where the fast axis is scanned using the triggering method while the slow axis is swept in do1d
4. 2D voltage sweep where both axes are scanned using the triggering method

### Experiment 1: 1D voltage sweep performed by the QDAC using do1d

In this example, the QDAC is parametrized to output a constant voltage which is swept using the do1d qcodes method. Here the OPX is simply measuring at each level, but the QUA program can easily be modified to complexify the sequence.

This method is similar to how any external DC voltage source would be integrated with the OPX.

In [ ]:
n_avg = 100  # Number of averaging loops


### OPX section
def qdac_1d_sweep(simulate=False):
    with program() as prog:
        i = declare(int)
        n = declare(int)
        data = declare(fixed)
        data_st = declare_stream()

        with infinite_loop_():
            if not simulate:
                pause()
            with for_(n, 0, n < n_avg, n + 1):
                # Wait 1ms before measuring (depends of the QDAC filter option)
                wait(1_000_000 // 4, "readout_element")
                # Measure for 10µs with the OPX - Can be replaced by dual_demod, demod or else
                measure("readout", "readout_element", None, integration.full("const", data, "out1"))
                # Send the result to the stream processing
                save(data, data_st)

        with stream_processing():
            # Average all the data and save the values into "data".
            data_st.buffer(n_avg).map(FUNCTIONS.average()).save_all("data")
    return prog


# Pass the readout length (in ns) to the class to convert the demodulated/integrated data into Volts
opx_instrument.readout_pulse_length(readout_len)
# Add the custom sequence to the OPX
opx_instrument.qua_program = qdac_1d_sweep(simulate=False)

### QDAC2 section
qdac2.reset()  # Reset the qdac parameters to start from a blank instrument
ch1 = qdac2.channel(2)  # Define the QDAC2 channel
# Set the current range ("high" or "low") and filter ("dc": 10Hz ,  "med": 10khz,  "high": 300kHz)
ch1.output_mode(range="low", filter="med")
# Set the slew rate (in V/s) to avoid transients when abrubtly stepping the voltage - Must be within [0.01; 2e7] V/s.
ch1.dc_slew_rate_V_per_s(1000)
Vg1 = ch1.dc_constant_V  # Define the voltage parameter for this channel

### Run the experiment
experiment1 = load_or_create_experiment("qdac_1d_sweep_do1d", sample_name)

start_time = time.time()
do1d(
    Vg1,
    -1.5,
    1.5,
    11,
    0.001,
    opx_instrument.resume,
    opx_instrument.get_measurement_parameter(),
    enter_actions=[opx_instrument.run_exp],
    exit_actions=[opx_instrument.halt],
    show_progress=True,
    # do_plot=True,
    exp=experiment1,
)
print(f"Elapsed time: {time.time() - start_time:.2f} s")
# Bring back the voltage to 0
Vg1(0)

### Experiment 2: 1D voltage sweep performed by the QDAC which is triggered by the OPX which is also measuring using do0d

In this example, the QDAC is parametrized to output the values defined in a pre-loaded voltage list. Stepping to the next value is done on the event of a digital marker sent by the OPX to one of the QDAC external trigger input. 
Here the OPX is simply triggering the QDAC and measuring at each level, but the QUA program can easily be modified to complexify the sequence.

This method is very similar to the previous one but significantly reduces the runtime since it avoids the communication time and data fetching at each voltage step. The experimental runtime is then given by the QDAC bandwidth and the integration time. The speed can also be further improved by increasing the QDAC bandwidth.

In [ ]:
n_avg = 100  # Number of averaging loops
# Voltage values in Volt
voltage_values1 = list(np.linspace(-1.5, 1.5, 101))


### OPX section
def qdac_1d_sweep_trig(simulate=False):
    with program() as prog:
        i = declare(int)
        n = declare(int)
        data = declare(fixed)
        data_st = declare_stream()

        with infinite_loop_():
            if not simulate:
                pause()
            with for_(n, 0, n < n_avg, n + 1):
                with for_(i, 0, i < len(voltage_values1), i + 1):
                    # Trigger the QDAC channel to output the next voltage level from the list
                    play("trig", "qdac_trigger1")
                    # Wait 1ms before measuring (depends of the QDAC filter option)
                    wait(1_000_000 // 4, "readout_element")
                    # Measure with the OPX
                    measure("readout", "readout_element", None, integration.full("const", data, "out1"))
                    # Send the result to the stream processing
                    save(data, data_st)

        with stream_processing():
            # Average all the data and save the values into "data".
            data_st.buffer(len(voltage_values1)).buffer(n_avg).map(FUNCTIONS.average()).save_all("data")
    return prog


# Pass the readout length (in ns) to the class to convert the demodulated/integrated data into Volts
opx_instrument.readout_pulse_length(readout_len)
# Set the setpoint for the loop performed on the OPX. Axis1 is the most inner non-averaging loop
opx_instrument.set_sweep_parameters("axis1", voltage_values1, "V", "Vg1")
# Add the custom sequence to the OPX
opx_instrument.qua_program = qdac_1d_sweep_trig(simulate=False)

### QDAC2 section
qdac2.reset()
ch1 = qdac2.channel(1)  # Define the QDAC2 channel
# Set the current range ("high" or "low") and filter ("dc": 10Hz ,  "med": 10khz,  "high": 300kHz)
ch1.output_mode(range="low", filter="med")
# Set the slew rate (in V/s) to avoid transients when abrubtly stepping the voltage - Must be within [0.01; 2e7] V/s.
ch1.dc_slew_rate_V_per_s(1000)
# Define the voltage list, stepping mode and dwell time
Vg1_list = ch1.dc_list(voltages=voltage_values1, stepped=True, dwell_s=5e-6)
# Set the trigger mode to external with input port "ext1"
Vg1_list.start_on_external(trigger=1)

### Run the experiment
experiment2 = load_or_create_experiment("qdac_1d_sweep_trig_do0d", sample_name)

start_time = time.time()
do0d(
    opx_instrument.run_exp,
    opx_instrument.resume,
    opx_instrument.get_measurement_parameter(),
    opx_instrument.halt,
    # do_plot=True,
    exp=experiment2,
)
print(f"Elapsed time: {time.time() - start_time:.2f} s")

# Bring the voltage back to 0
ch1.dc_constant_V(0)

### Experiment 3: 2D voltage sweep where the fast axis is scanned using the triggering method while the slow axis is swept in do1d

In this example, one channel of the QDAC is parametrized to step though a pre-loaded voltage list on the event of a digital marker provided by the OPX (fast axis). A second channel is set to output a constant voltage which will be scanned using the do1d qcodes method (slow axis).

The idea is for instance to acquire a charge stability map using a raster scan (for finding quantum dots).

This method is an extention of the previous ones showing how to sweep two voltage axes.

In [ ]:
n_avg = 100  # Number of averaging loops

# Voltage values in Volt
voltage_values1 = np.linspace(-0.4, 0.4, 51)
voltage_values2 = list(np.linspace(-1.5, 1.5, 51))

### QDAC2 section
qdac2.reset()  # Reset the qdac parameters
# Channel 1
ch1 = qdac2.channel(1)  # Define the QDAC2 channel
ch1.output_mode(range="low", filter="med")
# Set the slew rate (in V/s) to avoid transients when abrubtly stepping the voltage - Must be within [0.01; 2e7] V/s.
ch1.dc_slew_rate_V_per_s(1000)
# Define the voltage list, stepping mode and dwell time
Vg1_list = ch1.dc_list(voltages=voltage_values1, stepped=True, dwell_s=5e-6)
# Set the trigger mode to external with input port "ext1"
Vg1_list.start_on_external(trigger=1)
# Channel 2
ch2 = qdac2.channel(2)
ch2.output_mode(range="low", filter="med")
ch2.dc_slew_rate_V_per_s(1000)
Vg2 = ch2.dc_constant_V  # Define the voltage parameter for this channel


### OPX section
def qdac_opx_combined(simulate=False):
    with program() as prog:
        i = declare(int)
        n = declare(int)
        data = declare(fixed)
        data_st = declare_stream()

        with infinite_loop_():
            if not simulate:
                pause()

            with for_(i, 0, i < len(voltage_values1), i + 1):
                # Trigger the QDAC channel to output the next voltage level from the list
                play("trig", "qdac_trigger1")
                # Wait 1ms before measuring (depends of the QDAC filter option)
                wait(1_000_000 // 4, "readout_element")
                # In this example the averaging is done on the most inner loop (single point averaging)
                with for_(n, 0, n < n_avg, n + 1):
                    # Measure with the OPX
                    measure("readout", "readout_element", None, integration.full("const", data, "out1"))
                    # Send the result to the stream processing
                    save(data, data_st)
            # Bring the voltage back to zero
            ramp_to_zero("gate")

        with stream_processing():
            # Average all the data and save the values into "data".
            data_st.buffer(n_avg).map(FUNCTIONS.average()).buffer(len(voltage_values1)).save_all("data")
    return prog


# Pass the readout length (in ns) to the class to convert the demodulated/integrated data into Volts
opx_instrument.readout_pulse_length(readout_len)
# Add the custom sequence to the OPX
opx_instrument.qua_program = qdac_opx_combined(simulate=False)
# Axis1 is the most inner non-averaging loop
opx_instrument.set_sweep_parameters("axis1", voltage_values1, "V", "Vg1")

### Run the experiment
experiment3 = load_or_create_experiment("Combined_2D_sweep_do1d", sample_name)

start_time = time.time()
do1d(
    Vg2,
    voltage_values2[0],
    voltage_values2[-1],
    len(voltage_values2),
    0.001,
    opx_instrument.resume,
    opx_instrument.get_measurement_parameter(),
    enter_actions=[opx_instrument.run_exp],
    exit_actions=[opx_instrument.halt],
    show_progress=True,
    # do_plot=True,
    exp=experiment3,
)
print(f"Elapsed time: {time.time() - start_time:.2f} s")
# Bring the voltage back to 0
ch1.dc_constant_V(0)
Vg2(0)

### Experiment 4: 2D voltage sweep where both axes are scanned using the triggering method

In this example, two channels of the QDAC are parametrized to step though two pre-loaded voltage lists on the event of two digital markers provided by the OPX (connected to ext1 and ext2). This method allows the fast acquisition of a 2D voltage map and the data can be fetched from the OPX in real time to enable live plotting (this assumes that the averaging is performed on the most outer loop and the program is only to be executed once).

The speed can also be further improved by removing the live plotting and increasing the QDAC bandwidth.

In [ ]:
%matplotlib qt
n_avg = 10  # Number of averaging loops

# Voltage values in Volt
voltage_values1 = list(np.linspace(-0.4, 0.4, 51))
voltage_values2 = list(np.linspace(-1.5, 1.5, 21))

### QDAC2 section
qdac2.reset()  # Reset the qdac parameters
# Channel 1
ch1 = qdac2.channel(1)  # Define the QDAC2 channel
ch1.output_mode(range="low", filter="med")
# Set the slew rate (in V/s) to avoid transients when abrubtly stepping the voltage - Must be within [0.01; 2e7] V/s.
ch1.dc_slew_rate_V_per_s(1000)
# Define the voltage list, stepping mode and dwell time
Vg1_list = ch1.dc_list(voltages=voltage_values1, stepped=True, dwell_s=5e-6)
# Set the trigger mode to external with input port "ext1"
Vg1_list.start_on_external(trigger=1)
# Channel 2
ch2 = qdac2.channel(2)  # Define the QDAC2 channel
ch2.output_mode(range="low", filter="med")
ch2.dc_slew_rate_V_per_s(1000)
# Define the voltage list, stepping mode and dwell time
Vg2_list = ch2.dc_list(voltages=voltage_values2, stepped=True, dwell_s=5e-6)
# Set the trigger mode to external with input port "ext2"
Vg2_list.start_on_external(trigger=2)


### OPX section
def qdac_opx_combined(simulate=False):
    with program() as prog:
        i = declare(int)
        j = declare(int)
        n = declare(int)
        data = declare(fixed)
        data_st = declare_stream()

        # No need for the infinite loop here since the program will only be executed once.
        # When using live plotting, the infinite_loop will prevent the program from ending because it blocks the python terminal
        # with infinite_loop_():
        if not simulate:
            pause()
        # In this example the averaging is done on the most outer loop to enable live plotting
        with for_(n, 0, n < n_avg, n + 1):
            with for_(i, 0, i < len(voltage_values2), i + 1):
                # Trigger the QDAC channel to output the next voltage level from the list
                play("trig", "qdac_trigger2")
                with for_(j, 0, j < len(voltage_values1), j + 1):
                    # Trigger the QDAC channel to output the next voltage level from the list
                    play("trig", "qdac_trigger1")
                    # Wait 1ms before measuring (depends of the QDAC filter option)
                    wait(1_000_000 // 4, "readout_element")
                    # Measure with the OPX
                    measure("readout", "readout_element", None, integration.full("const", data, "out1"))
                    # Send the result to the stream processing
                    save(data, data_st)

        with stream_processing():
            # When averaging on the most outer loop, if the program is meant to run once (do0d),
            # then the .average() method can be used for live plotting purposes
            data_st.buffer(len(voltage_values1)).buffer(len(voltage_values2)).average().save_all("data")
    return prog


# Pass the readout length (in ns) to the class to convert the demodulated/integrated data into Volts
opx_instrument.readout_pulse_length(readout_len)
# Add the custom sequence to the OPX
opx_instrument.qua_program = qdac_opx_combined(simulate=False)
# Axis1 is the most inner non-averaging loop
opx_instrument.set_sweep_parameters("axis1", voltage_values1, "V", "Vg1")
opx_instrument.set_sweep_parameters("axis2", voltage_values2, "V", "Vg2")

### Run the experiment
experiment4 = load_or_create_experiment("Combined_2D_sweep_do0d", sample_name)

start_time = time.time()
# Compile the QUA program and execute it
opx_instrument.run_exp()
# Exit the pause() statement and start the sequence
opx_instrument.resume()
# Uncomment to fetch the data in real-time and plot them
# opx_instrument.live_plotting(["data"])
# Update the data counter to save the last dataset to the qcodes database
opx_instrument.counter = n_avg
# Store the results in the qcodes database
do0d(
    opx_instrument.get_measurement_parameter(),
    do_plot=True,
    exp=experiment4,
)

print(f"Elapsed time: {time.time() - start_time:.2f} s")
# Bring the voltage back to 0
ch1.dc_constant_V(0)
ch2.dc_constant_V(0)